## Importações

In [1]:
import requests
import pandas as pd
from funcoes import *
import pickle

In [2]:
r = requests.get("https://mof.tech.northwestern.edu/mofs.json")

print(r)

<Response [200]>


In [3]:
data = r.json()['results']

In [4]:
df = pd.DataFrame.from_dict(data)

## Tratamento de dados

In [5]:
df = funcao_extrair_features_isotherms(df)

In [6]:
df_mof = df.iloc[100:130,:]
df_mof = extrair_cif(df_mof,minimo=3)

GlobalSymmetryFeatures:   0%|          | 0/30 [00:00<?, ?it/s]

ElementProperty:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
# Assuming your dataframe is named df
with open('dataframe.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import pickle